## Simple transformation

In [1]:
import apache_beam as beam

with beam.Pipeline() as p:
    lines = (
        p | 'Create' >> beam.Create(['one', 'two', 'three', 'four'])
          | 'Uppercase' >> beam.Map(str.title)
          | 'Print' >> beam.Map(print)
    )



One
Two
Three
Four


## The pipe | is actually just an operator overload to call the apply method of the pipeline

In [2]:
import apache_beam as beam

with beam.Pipeline() as p:
        lines = p | 'Create' >> beam.Create(['one', 'two', 'three', 'four'])
        (lines 
            .apply(beam.Map(str.title), label = 'titlecase') 
            .apply(beam.Map(print), label='print')
        )

One
Two
Three
Four


## Read from CSV and use Map.
<br>
<details><summary>Click for <b>Java</b></summary>
<p> 
    
```java

package com.mypackage;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.PipelineResult;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.values.TypeDescriptors;
import org.apache.beam.sdk.transforms.MapElements;
import org.apache.beam.sdk.io.TextIO;


public class Simple1 {
    public static void main(String[] args) {
        Pipeline p = Pipeline.create();

        String regionsInputFileName = "regions.csv";
        String regionsOutputFileName = "output/regions";

        PCollection<String> regions = p
            .apply("Read", TextIO.read().from(regionsInputFileName))
            .apply("Parse", MapElements.into(TypeDescriptors.strings()).via((String element) -> element + "*"));
        regions.apply("Write", TextIO.write().to(regionsOutputFileName));
        p.run().waitUntilFinish();
    }
}
                       
```
</p>
</details>
<details><summary>Click for <b>Python</b></summary>
<p>

<p>
    
```python
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText

def splitregion(line):
    #1,North
    return tuple(line.split(','))

regionsfilename = 'regions.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(regionsfilename)
#          | 'Parse' >> beam.Map(lambda x : tuple(x.split(',')))
          | 'Parse' >> beam.Map(splitregion)
          | 'Transform' >> beam.Map(lambda x : (int(x[0]) * 10, x[1].upper()))
          | 'Print' >> beam.Map(print)
    )
    #p.run()
```
</p>

</details>

In [3]:
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText

def splitregion(line):
    #1,North
    return tuple(line.split(','))

regionsfilename = 'regions.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(regionsfilename)
#          | 'Parse' >> beam.Map(lambda x : tuple(x.split(',')))
          | 'Parse' >> beam.Map(splitregion)
          | 'Transform' >> beam.Map(lambda x : (int(x[0]) * 10, x[1].upper()))
          | 'Print' >> beam.Map(print)
    )
    #p.run()

(10, 'EASTERN')
(20, 'WESTERN')
(30, 'NORTHERN')
(40, 'SOUTHERN')


## Read from CSV and use ParDo

In [4]:
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText

class RegionParseTuple(beam.DoFn):
    def process(self, element):
        regionid, regionname = element.split(',')
        #return [(int(regionid), regionname)] # ParDo's need to return a list
        yield (int(regionid), regionname) # Can also use yield instead of returning a list

regionsfilename = 'regions.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(regionsfilename)
          | 'Parse' >> beam.ParDo(RegionParseTuple())
          | 'Write' >> WriteToText('regions.out')
    )


In [ ]:
! cat regions.out*

## Template showing a full program that can read the command line args

In [ ]:
"""A template to import the default package and parse the arguments"""

from __future__ import absolute_import

import argparse
import logging
import re, os

#from past.builtins import unicode

import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

class RegionParseTuple(beam.DoFn):
    def process(self, element):
        regionid, regionname = element.split(',')
        yield (int(regionid), regionname) # Can also use yield instead of returning a list

def run(argv=None, save_main_session=True):
  """Main entry point; defines and runs the wordcount pipeline."""
  projectid = os.environ.get('PROJECT')
  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--input',
      dest='input',
      default=f'gs://{projectid}/regions.csv',
      help='Input file to process.')
  parser.add_argument(
      '--output',
      dest='output',
      default = f'gs://{projectid}/regions_output',      
      help='Output file to write results to.')
  known_args, pipeline_args = parser.parse_known_args(argv)

  # We use the save_main_session option because one or more DoFn's in this
  # workflow rely on global context (e.g., a module imported at module level).
  pipeline_options = PipelineOptions(pipeline_args)
  pipeline_options.view_as(SetupOptions).save_main_session = save_main_session

  # The pipeline will be run on exiting the with block.
  with beam.Pipeline(options=pipeline_options) as p:
    lines = p | 'Read' >> ReadFromText(known_args.input)
    records = lines | 'Split' >> beam.ParDo(RegionParseTuple())
    uppercase = records | 'Uppercase' >> beam.Map(lambda x : (int(x[0]), x[1].upper()))
    uppercase | 'Write' >> WriteToText(known_args.output)

if __name__ == '__main__':
  logging.getLogger().setLevel(logging.INFO)
  run()



## Example of how to create a split ParDo with multiple outputs

In [ ]:
import apache_beam as beam
from apache_beam import pvalue
from apache_beam.io import ReadFromText

class OddEvenRegionParseTuple(beam.DoFn):
    def process(self, element):
        regionid, regionname = element.split(',')
        if int(regionid) % 2 == 0:
            yield pvalue.TaggedOutput('Even', (int(regionid), regionname, 'Even'))
        else:
            yield pvalue.TaggedOutput('Odd', (int(regionid), regionname, 'Odd'))

regionsfilename = 'regions.csv'
with beam.Pipeline() as p:
    lines = p | 'Read' >> ReadFromText(regionsfilename)
    evens, odds = lines | 'Parse' >> beam.ParDo(OddEvenRegionParseTuple()).with_outputs("Even", "Odd")
    
    print('Evens')
    evens | 'Print Evens' >> beam.Map(print)

    print('Odds')
    odds | 'Print Odds' >> beam.Map(print)
    
  

## Example of branching

In [ ]:
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText

class RegionParseTuple(beam.DoFn):
    def process(self, element):
        regionid, regionname = element.split(',')
        #return [(int(regionid), regionname)] # ParDo's need to return a list
        yield (int(regionid), regionname) # Can also use yield instead of returning a list

regionsfilename = 'regions.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(regionsfilename)
          | 'Parse' >> beam.ParDo(RegionParseTuple())
    )
    # Branch 1
    (regions 
         | 'Lowercase regions' >> beam.Map(lambda x : (x[0] * 100, x[1].lower()))
         | 'Write' >> WriteToText('regions2.out')
    )
    # Branch 2
    (regions 
         | 'Uppercase regions' >> beam.Map(lambda x : (x[0] * 10, x[1].upper()))
         | 'Print' >> beam.Map(print)
    )



In [ ]:
! cat regions2*

## WithKeys

In [ ]:
import apache_beam as beam
from apache_beam.io import ReadFromText

class TerritoryParseTuple(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield (int(territoryid), territoryname, int(regionid))

territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    territories = (
                  p | 'Read Territories' >> ReadFromText(territoriesfilename)
                    | 'Parse Territories' >> beam.ParDo(TerritoryParseTuple())
                    | 'Territories With Keys' >> beam.util.WithKeys(lambda x : x[2])
#                    | 'With Keys Manually' >> beam.Map(lambda x : (x[2], x))
                  )
#    territories | 'Print KV' >> beam.Map(print)
#    territories | beam.util.Keys() | 'Print Keys' >> beam.Map(print)
#     territories | beam.util.Values() | 'Print Values' >> beam.Map(print)




## GroupByKey

In [ ]:
import apache_beam as beam
from apache_beam.io import ReadFromText

class TerritoryParseTuple(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield (int(territoryid), territoryname, int(regionid))

territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    territories = (
                  p | 'Read Territories' >> ReadFromText('territories.csv')
                    | 'Parse Territories' >> beam.ParDo(TerritoryParseTuple())
                    | 'Territories With Keys' >> beam.util.WithKeys(lambda x : x[2])
                    | 'Group Territories' >> beam.GroupByKey() 
#                    | 'Print Territories' >> beam.Map(print)
                    | 'Write' >> WriteToText('territories_group.out')
                  )


In [ ]:
! ls territories_group.out*
#! cat territories_group.out

## Flatten

In [ ]:
import apache_beam as beam

with beam.Pipeline() as p:
    lines1 = p | 'Create 1' >> beam.Create(['one', 'two', 'three', 'four'])
    lines2 = p | 'Create 2' >> beam.Create(['alpha', 'beta', 'gamma', 'delta'])

    merged = ((lines1, lines2) | 'Merge PCollections' >> beam.Flatten())
    merged | beam.Map(print)


## Combine

In [ ]:
import apache_beam as beam

with beam.Pipeline() as p:
    data = (
        p | 'Create' >> beam.Create([('a', 10), ('a', 20), ('b', 30), ('b', 40), ('c', 50), ('a', 60)])
          | 'Combine' >> beam.CombinePerKey(sum)
          | 'Print' >> beam.Map(print)
    )


## Custom Combine Function

In [6]:
#mport apache_beam as beam

class CustomCombine(beam.CombineFn):

  def create_accumulator(self):
    return dict()

  def add_input(self, accumulator, input):
    k, v = input
    x, y, z = accumulator.get(k, (0, 0, 0))

    # take the max for the first element of the tuple and sum the second element and count for the third
    accumulator[k] = (v[0] if v[0] > x else x, y + v[1], z + 1)
    return accumulator

  def merge_accumulators(self, accumulators):
    merged = dict()
    for accum in accumulators:
      for k, v in accum.items():
        x, y, z = merged.get(k, (0, 0, 0))
        merged[k] = (v[0] if v[0] > x else x, y + v[1], z + v[2])
    return merged

  def extract_output(self, accumulator):
    # return the max, the sum, the count and the average for the key
    return {k : (v[0], v[1], v[2], v[1]/v[2]) for k, v in accumulator.items()}
    return accumulator
    

with beam.Pipeline() as p:
    data = (
        p | 'Create' >> beam.Create([('a', (1, 10)), ('a', (2, 20)), 
                                     ('b', (3, 30)), ('c', (5, 50)), 
                                     ('b', (4, 40)), ('a', (6, 60))])
          | 'Combine' >> beam.CombineGlobally(CustomCombine())
          | 'Print' >> beam.Map(print)
    )


{'a': (6, 90, 3, 30.0), 'b': (4, 70, 2, 35.0), 'c': (5, 50, 1, 50.0)}


## Map vs FlatMap

In [ ]:
import apache_beam as beam

with beam.Pipeline() as pipeline:
  plants = (
      pipeline
      | 'Gardening plants' >> beam.Create(['Strawberry,Carrot,Eggplant','Tomato,Potato'])
#      | 'Split words' >> beam.Map(lambda x : x.split(','))
      | 'Split words' >> beam.FlatMap(lambda x : x.split(','))
      | beam.Map(print))

## Parsing the line as a tuple

In [ ]:
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText

class TerritoryParseTuple(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield(int(territoryid), territoryname, int(regionid))

class StartsWithSFilter(beam.DoFn):
    def process(self, element):
        if element[1].startswith('S'):
            yield element
            
territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(territoriesfilename)
          | 'Parse' >> beam.ParDo(TerritoryParseTuple())
          | 'Filter 1' >> beam.Filter(lambda x : x[2] % 2 == 0)
#          | 'Filter 2' >> beam.Filter(lambda x : x[1].startswith('S'))
          | 'Filter 2' >> beam.ParDo(StartsWithSFilter())
          | 'Print' >> beam.Map(print)
#          | 'Write' >> WriteToText('regions.out')
    )


In [ ]:
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText

class TerritoryParseDict(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield {'territoryid' : int(territoryid), 'territoryname' : territoryname, 'regionid'  : int(regionid)}

class StartsWithSFilter(beam.DoFn):
    def process(self, element):
        if element[1].startswith('S'):
            yield element
            
territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(territoriesfilename)
          | 'Parse' >> beam.ParDo(TerritoryParseDict())
          | 'Filter 1' >> beam.Filter(lambda x : x['regionid'] % 2 == 0)
          | 'Filter 2' >> beam.Filter(lambda x : x['territoryname'].startswith('S'))
          | 'Print' >> beam.Map(print)
#          | 'Write' >> WriteToText('regions.out')
    )


In [ ]:
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
import typing


class Territory(typing.NamedTuple):
    territoryid: int
    territoryname: str
    regionid: int
beam.coders.registry.register_coder(Territory, beam.coders.RowCoder)
        
class TerritoryParseClass(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield Territory(int(territoryid), territoryname, int(regionid))

class StartsWithSFilter(beam.DoFn):
    def process(self, element):
        if element[1].startswith('S'):
            yield element
            
territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    regions = (
        p | 'Read' >> ReadFromText(territoriesfilename)
          | 'Parse' >> beam.ParDo(TerritoryParseClass())
          | 'Filter 1' >> beam.Filter(lambda x : x.regionid % 2 == 0)
          | 'Filter 2' >> beam.Filter(lambda x : x.territoryname.startswith('S'))
          | 'Print' >> beam.Map(print)
#          | 'Write' >> WriteToText('regions.out')
    )


## Side Inputs with a single value

In [ ]:
import apache_beam as beam
from apache_beam.pvalue import AsSingleton, AsDict
from apache_beam.io import ReadFromText

class TerritoryParseTuple(beam.DoFn):
    # split territory into KV pair of (regionid, (territoryid, territoryname))
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield(int(territoryid), territoryname, int(regionid))

class LookupRegion(beam.DoFn):
    def process(self, element, uppercase = 0):
        lookuptable = {1:'North', 2:'South', 3:'East', 4:'West'}
        territoryid, territoryname, regionid = element
        region = lookuptable.get(regionid, 'No Region')
        if uppercase == 1:
            region = region.upper()
        yield(territoryid, territoryname, regionid, region)
        
with beam.Pipeline() as p:
    territories =  (
        p | 'Read Territories' >> ReadFromText('territories.csv')
          | 'Parse Territories' >> beam.ParDo(TerritoryParseTuple())
    )
    
    # This is contrived, but let's just calculate a value that we can pass as a single side input to LookupRegion
    minregion = (
        territories
          | 'Extract RegionID' >> beam.Map(lambda x : x[2])
          | 'MaxTerritories' >> beam.CombineGlobally(lambda elements: min(elements or [None]))
    )

    lookup = (
        territories
#          | beam.ParDo(LookupRegion(), uppercase = 0 ) 
          | beam.ParDo(LookupRegion(), uppercase = beam.pvalue.AsSingleton(minregion) ) 
#          | beam.ParDo(LookupRegion(), uppercase = minregion ) 
          | 'Print Loopup' >> beam.Map(print)
    )

#    maxregion | 'Print Min' >> beam.Map(print)


## Side input that is a lookup list

In [ ]:
import apache_beam as beam
from apache_beam.pvalue import AsList
from apache_beam.io import ReadFromText

class RegionParseDict(beam.DoFn):
    def process(self, element):
        regionid, regionname = element.split(',')
        yield {'regionid': int(regionid), 'regionname': regionname.title()}

class TerritoryParseTuple(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield(int(territoryid), territoryname, int(regionid))
        
                
class LookupRegion(beam.DoFn):
    def process(self, element, lookuptable = [{'regionid':1, 'regionname':'North'}, {'regionid':2, 'regionname':'South'}]):
        territoryid, territoryname, regionid = element
        # Becase the regions PCollection is a different shape, use the following comprehension to make it easier to do a lookup
        lookup = {e['regionid'] : e['regionname'] for e in lookuptable } # {1:'North', 2:'South'}
        yield(territoryid, territoryname, regionid, lookup.get(regionid, 'No Region'))

with beam.Pipeline() as p:
    regions = (
        p | 'Read Regions' >> ReadFromText('regions.csv')
          | 'Parse Regions' >> beam.ParDo(RegionParseDict())
#          | 'Print Regions' >> beam.Map(print)
    )

    territories =  (
        p | 'Read Territories' >> ReadFromText('territories.csv')
          | 'Parse Territories' >> beam.ParDo(TerritoryParseTuple())
#          | 'Print Territories' >> beam.Map(print)
    )
    
    lookup = (
        territories
        | beam.ParDo(LookupRegion(), lookuptable = beam.pvalue.AsList(regions))
        | 'Print Loopup' >> beam.Map(print)
    )
        


## Create a nested repeating output
### First create a dataset. Here is python code for the equivalent bq command of bq mk dataflow


In [18]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set dataset_id to the ID of the dataset to create.
PROJECT_ID = 'qwiklabs-gcp-04-4cf93802c378'
dataset_id = f"{PROJECT_ID}.dataflow" #.format(client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# TODO(developer): Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))



Created dataset qwiklabs-gcp-04-4cf93802c378.dataflow1


## Run the following in a bq query window to create the table

In [29]:
# 
create table dataflow.region_territory
(regionid NUMERIC
,regionname STRING
,territories ARRAY<STRUCT<territoryid NUMERIC, territoryname STRING>>)

BigQuery error in mk operation: Invalid schema entry:
territories:ARRAY<STRUCT<territoryid:NUMERIC


## Need to manually create a schema for a nested repeating, cannot use a simple string

In [69]:
from apache_beam.io.gcp.internal.clients import bigquery as bq
region_territory_schema = bq.TableSchema()
regionid = bq.TableFieldSchema()
regionid.name='regionid'
regionid.type='string'
regionid.mode='required'
region_territory_schema.fields.append(regionid)
regionname = bq.TableFieldSchema()
regionname.name='regionname'
regionname.type='string'
regionname.mode='required'
region_territory_schema.fields.append(regionname)


# A nested field
territories = bq.TableFieldSchema()
territories.name='territores'
territories.type='record'
territories.mode='nullable'


territoryid = bq.TableFieldSchema()
territoryid.name='territoryid'
territoryid.type='string'
territoryid.mode='required'
territories.fields.append(territoryid)
territoryname = bq.TableFieldSchema()
territoryname.name='territoryname'
territoryname.type='string'
territoryname.mode='required'
territories.fields.append(territoryname)

region_territory_schema.fields.append(territories)

#print(region_territory_schema)

## The code here is tricky: 
### First parse the two tables into tuples, (regionid, regionname) & (regionid, {'territoryid':territoryid, 'territoryname':territoryname})


In [99]:
import apache_beam as beam
from apache_beam.io import ReadFromText

class RegionParseTuple(beam.DoFn):
    def process(self, element):
        regionid, regionname = element.split(',')
        yield (int(regionid), regionname) # Can also use yield instead of returning a list

class TerritoryParseTuple(beam.DoFn):
    # split territory into KV pair of (regionid, (territoryid, territoryname))
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield(int(regionid), {'territoryid': int(territoryid), 'territoryname':territoryname})

regionsfilename = 'regions.csv'
territoriesfilename = 'territories.csv'
PROJECT_ID = 'qwiklabs-gcp-04-4cf93802c378'

with beam.Pipeline() as p:
    regions = (
              p | 'Read Regions' >> ReadFromText(regionsfilename)
                | 'Parse Regions' >> beam.ParDo(RegionParseTuple())
                #| 'Print Regions' >> beam.Map(print)
              )
        
    territories = (
                  p | 'Read Territories' >> ReadFromText('territories.csv')
                    | 'Parse Territories' >> beam.ParDo(TerritoryParseTuple())
                    #| 'Print Territories' >> beam.Map(print)
                  )
    nested = ( 
        {'regions':regions, 'territories':territories} 
              | 'Nest territories into regions' >> beam.CoGroupByKey()
              | 'Reshape to dict' >> beam.Map(lambda x : {'regionid': x[0], 'regionname': x[1]['regions'][0], 
                                                        'territories': x[1]['territories']})
    )
    nested | 'Write nested region_territory to BQ' >> beam.io.WriteToBigQuery('region_territory', dataset = 'dataflow'
                                                                             , project = PROJECT_ID
                                                                             , schema = region_territory_schema
                                                                             , method = 'STREAMING_INSERTS'
                                                                             )
#    nested | 'Print' >> beam.Map(print)
             
#help(beam.io.WriteToBigQuery)    
        

## Simulate an Outer Join with CoGroup

In [ ]:
import apache_beam as beam
from apache_beam.io import ReadFromText

class RegionParseDict(beam.DoFn):
    def process(self, element):
        regionid, regionname = element.split(',')
        yield {'regionid':int(regionid), 'regionname':regionname.title()}

class TerritoryParseDict(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield {'territoryid':int(territoryid), 'territoryname' : territoryname, 'regionid':int(regionid)}

class UnnestCoGrouped(beam.DoFn):
    def process(self, item, child_pipeline, parent_pipeline):
        k, v = item
        child_dict = v[child_pipeline]
        parent_dict = v[parent_pipeline]
        for child in child_dict:
            try:
                child.update(parent_dict[0])
                yield child
            except IndexError:
                yield child

class LeftJoin(beam.PTransform):
    def __init__(self, parent_pipeline_name, parent_data, parent_key, child_pipeline_name, child_data,  child_key):
        self.parent_pipeline_name = parent_pipeline_name
        self.parent_data = parent_data
        self.parent_key = parent_key
        self.child_pipeline_name = child_pipeline_name
        self.child_data = child_data
        self.child_key = child_key

    def expand(self, pcols):
        def _format_as_common_key_tuple(child_dict, child_key):
            return (child_dict[child_key], child_dict)

        return ({
                pipeline_name: pcol1 | f'Convert to ({self.parent_key} = {self.child_key}, object) for {pipeline_name}' 
                >> beam.Map(_format_as_common_key_tuple, self.child_key)
                for (pipeline_name, pcol1) in pcols.items()}
                | f'CoGroupByKey {pcols.keys()}' >> beam.CoGroupByKey()
                | 'Unnest Cogrouped' >> beam.ParDo(UnnestCoGrouped(), self.child_pipeline_name, self.parent_pipeline_name)
        )
        
regionsfilename = 'regions.csv'
territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    regions = (
              p | 'Read Regions' >> ReadFromText(regionsfilename)
                | 'Parse Regions' >> beam.ParDo(RegionParseDict())
                #| 'Print Regions' >> beam.Map(print)
              )
        
    territories = (
                  p | 'Read Territories' >> ReadFromText('territories.csv')
                    | 'Parse Territories' >> beam.ParDo(TerritoryParseDict())
                    #| 'Print Territories' >> beam.Map(print)
                  )

    leftjoin = {'regions':regions, 'territories':territories} | LeftJoin('regions', regions, 'regionid', 'territories', territories, 'regionid')
    leftjoin | 'Print Left Join' >> beam.Map(print)




(1, north) (2, south)

10, 1    
11, 1
12, 2
13, 2
14, 1


(1, [(10, 1), (11, 1), (14,1)], [(1, north)])
(2, [(12, 2), (13,2)], [(2, South)])


all region 1 end up on some worker from both the regions and territories


[ pcol1, pcol2] | 

## BeamSQL

In [ ]:
! docker images
#! docker pull apache/beam_java11_sdk 
#! docker pull apache/beam_java8_sdk 

In [ ]:
# %%writefile beamsql1.py
# This code is not running in the notebook
# This example just uses a basic Row object
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam import coders
from apache_beam.transforms.sql import SqlTransform

import typing
import json

with beam.Pipeline() as p:
    parent = (
            p | 'Create Parent' >> beam.Create([(1, 'One'), (2, 'Two')])
              | 'Map Parent' >> beam.Map(lambda x : beam.Row(parent_id = x[0], parent_name = x[1]))
    )

    child = (
            p | 'Create Child' >> beam.Create([('Uno', 1), ('Due', 2), ('Eins', 1), ('Una', 1), ('Dos', 2)])
              | 'Map Child' >> beam.Map(lambda x : beam.Row(child_name = x[0], parent_id = x[1]))
    )
    
    ( {'parent': parent, 'child' : child} 
         | SqlTransform("""
             SELECT p.parent_id, p.parent_name, c.child_name 
             FROM parent as p 
             INNER JOIN child as c ON p.parent_id = c.parent_id
             """)
        | 'Map Join' >> beam.Map(lambda x : f'{x.parent_id} {x.parent_name} {x.child_name}')
        | 'Print Join' >> beam.Map(print)
        )

#     parent | 'print parent' >> beam.Map(print)
#     child  | 'print child' >> beam.Map(print)


In [ ]:
# %%writefile beamsql2.py
# This code is not running in the notebook
# This example uses a simple class to handle the schemas
import apache_beam as beam
from apache_beam import coders
from apache_beam.transforms.sql import SqlTransform

import typing


class Parent(typing.NamedTuple):
    parent_id: int
    parent_name: str
beam.coders.registry.register_coder(Parent, beam.coders.RowCoder)

class Child(typing.NamedTuple):
    child_name: str
    parent_id: int
beam.coders.registry.register_coder(Child, beam.coders.RowCoder)
        

with beam.Pipeline() as p:
    parent = (
            p | 'Create Parent' >> beam.Create([(1, 'One'), (2, 'Two')])
              | 'Map Parent' >> beam.Map(lambda x : Parent(parent_id = x[0], parent_name = x[1])).with_output_types(Parent)
              | 'Map for Print' >> beam.Map(print)
    )

    child = (
            p | 'Create Child' >> beam.Create([('Uno', 1), ('Due', 2), ('Eins', 1), ('Una', 1), ('Dos', 2)])
              | 'Map Child' >> beam.Map(lambda x : Child(child_name = x[0], parent_id = x[1])).with_output_types(Child)
#               | 'SQL Child' >> SqlTransform("""SELECT 10 * parent_id as parent_id, upper(child_name) as child_name from PCOLLECTION""")
#               | 'Print Map' >> beam.Map(lambda x : f'{x.parent_id} = {x.child_name}')
              | 'SQL Child' >> SqlTransform("""SELECT parent_id, count(*) as cnt from PCOLLECTION GROUP BY parent_id""")
              | 'Map for Print' >> beam.Map(lambda x : f'{x.parent_id} = {x.cnt}')
              | 'Print SQL' >> beam.Map(print)
    )

In [ ]:
%%writefile beamsql3.py
# This code is not running in the notebook
# This example is like example 2 but for a real file 
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam import coders
from apache_beam.transforms.sql import SqlTransform

import typing
import json

class Territory(typing.NamedTuple):
    territoryid: int
    territoryname: str
    regionid: int

coders.registry.register_coder(Territory, coders.RowCoder)
        
@beam.typehints.with_output_types(Territory)
class TerritoryParseClass(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield Territory(int(territoryid), territoryname.title(), int(regionid))

        
territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    territories = (
                  p | 'Read Territories' >> ReadFromText('territories.csv')
#                    | 'Parse Territories' >> beam.ParDo(TerritoryParseClass()).with_output_types(Territory)
                    | 'Parse Territories' >> beam.ParDo(TerritoryParseClass())
                    | 'SQL Territories' >> SqlTransform("""SELECT regionid, count(*) as `cnt` FROM PCOLLECTION GROUP BY regionid""")
                    | 'Map Territories for Print' >> beam.Map(lambda x : f'{x.regionid} - {x.cnt}')
                    | 'Print SQL' >> beam.Map(print)
                    )
    
#https://www.youtube.com/watch?v=zx4p-UNSmrA

In [ ]:
territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    territories = (
                  p | 'Read Territories' >> ReadFromText('territories.csv')
                    | 'Parse Territories' >> beam.ParDo(TerritoryParseClass())
                    | 'SQL Territories' >> SqlTransform("""SELECT regionid, count(*) as `cnt` FROM PCOLLECTION GROUP BY regionid""")
                    )

                    territories | 'Write to BQ' >> beam.WriteToBQ()
                    territories | 'Write TO File' >> beam.WriteToText()
#    p.run()


In [ ]:
territoriesfilename = 'territories.csv'
with beam.Pipeline() as p:
    territories = (
                  p | 'Read Territories' >> ReadFromText('territories.csv')
                    | 'Parse Territories' >> beam.ParDo(TerritoryParseClass())
                  )
                  
                (territories | 'SQL Territories' >> SqlTransform("""SELECT regionid, count(*) as `cnt` FROM PCOLLECTION GROUP BY regionid""")
                            | 'Write to BQ' >> beam.WriteToBQ()
                )
                (territories | 'SQL Territories2' >> SqlTransform("""SELECT territoryid, count(*) as `cnt` FROM PCOLLECTION GROUP BY regionid""")
                            | 'Write to BQ' >> beam.WriteToBQ()
                    territories | 'Write TO File' >> beam.WriteToText()
                )
#    p.run()


## DoFn Lifecycle

In [ ]:
import apache_beam as beam
from apache_beam.pvalue import AsSingleton, AsDict
from apache_beam.io import ReadFromText

class TerritoryParseTuple(beam.DoFn):
    # split territory into KV pair of (regionid, (territoryid, territoryname))
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield(int(territoryid), territoryname, int(regionid))
        
                
class LookupRegion(beam.DoFn):
    def setup(self):
        self.lookup = {1:'North', 2:'South', 3:'East', 4:'West'}
        print('setup')
        
    def start_bundle(self):
        print('start bundle')
        
    def process(self, element, uppercase = 0):
        #lookuptable = {1:'North', 2:'South', 3:'East', 4:'West'}
        territoryid, territoryname, regionid = element
        region = self.lookup.get(regionid, 'No Region')
        if uppercase == 1:
            region = region.upper()
        yield(territoryid, territoryname, regionid, region)
        
    def finish_bundle(self):
        print('finish bundle')

    def teardown(self):
        print('teardown')
        del self.lookup
    

with beam.Pipeline() as p:
    territories =  (
        p | 'Read Territories' >> ReadFromText('territories.csv')
          | 'Parse Territories' >> beam.ParDo(TerritoryParseTuple())
    )
    
    lookup = (
        territories
        | beam.ParDo(LookupRegion(), uppercase = 1 ) 
        | 'Print Loopup' >> beam.Map(print)
    )
        


In [ ]:
import apache_beam as beam
from apache_beam.pvalue import AsList
from apache_beam.io import ReadFromText, WriteToText

class RegionParseDict(beam.DoFn):
    def process(self, element):
        regionid, regionname = element.split(',')
        yield {'regionid': int(regionid), 'regionname': regionname.title()}

class TerritoryParseTuple(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield(int(territoryid), territoryname, int(regionid))
        
class LookupRegion(beam.DoFn):
    def __init__(self):
        print('init')
        #self.lookup = {1:'north', 2:'south', 3:'east', 4:'west'}
        #self.init_semaphore = False

    def called_once(self, lookuptable):
        print('called_once')
        self.lookup = { e['regionid'] : e['regionname'] for e in lookuptable }
        self.init_semaphore = False

    def setup(self):
        print('setup')
        self.init_semaphore = True
        #self.lookup = {1:'NORTH', 2:'South', 3:'East', 4:'West'}

    def start_bundle(self):
        print('start bundle')

    def process(self, element, lookuptable = [{'regionid':1, 'regionname':'north'}, {'regionid':2, 'regionname':'south'}]):
        if self.init_semaphore:
            self.called_once(lookuptable)
        territoryid, territoryname, regionid = element
        yield(territoryid, territoryname, regionid, self.lookup.get(regionid, 'No Region'))

    def finish_bundle(self):
        print('finish bundle')

    def teardown(self):
        print('teardown')
        del self.lookup
        del self.init_semaphore
                


with beam.Pipeline() as p:

    regions = (
        p | 'Read Regions' >> ReadFromText('regions.csv')
          | 'Parse Regions' >> beam.ParDo(RegionParseDict())
    )

    territories =  (
        p | 'Read Territories' >> ReadFromText('territories.csv')
          | 'Spare Territories' >> beam.ParDo(TerritoryParseTuple())
    )
    
    lookup = (
        territories
        | beam.ParDo(LookupRegion(), lookuptable = beam.pvalue.AsList(regions))
        | 'Print Loopup' >> beam.Map(print)
    )
        


## Code below is not working

In [ ]:
import json
import logging

import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.transforms.sql import SqlTransform
#from collections import namedtuple
from typing import NamedTuple
from apache_beam import coders


RegionSchema = namedtuple("RegionSchema", ("regionid", "regionname"))
#coders.registry.register_coder(RegionSchema, coders.RowCoder)
class RegionSplitSchema(beam.DoFn):
    def process(self, element):
        regionid, regionname = element.split(',')
        yield RegionSchema(int(regionid), regionname.title())

#TerritorySchema = namedtuple("TerritorySchema", ("territoryid", "territoryname", "regionid"))
TerritorySchema = NamedTuple("TerritorySchema", [("territoryid", int), ("territoryname", str), ("regionid", int)])
coders.registry.register_coder(TerritorySchema, coders.RowCoder)
class TerritorySplitSchema(beam.DoFn):
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield TerritorySchema(int(territoryid), territoryname.title(), int(regionid))

        
# class TerritorySplitNamedTuple(beam.DoFn):
#     def process(self, element):
#         territoryid, territoryname, regionid = element.split(',')
#         yield {'territoryid':int(territoryid), 'territoryname' : territoryname, 'regionid':int(regionid)}

regionsfilename = 'regions.csv'
territoriesfilename = 'territories.csv'
print('Start')
with beam.Pipeline() as p:
#     regions = (
#               p | 'Read Regions' >> ReadFromText(regionsfilename)
#                 | 'Split Regions' >> beam.ParDo(RegionSplitSchema()).with_output_types(RegionSchema)
#               )
        
    territories = (
                  p | 'Read Territories' >> ReadFromText('territories.csv')
                    | 'Split Territories' >> beam.ParDo(TerritorySplitSchema()) #.with_output_types(TerritorySchema)
#                    | beam.Map(lambda x: TerritorySchema(x.territoryid, x.territoryname, x.regionid)).with_output_types(TerritorySchema)
#         Map(lambda x: PythonSchema(f_int32=x)).with_output_types(PythonSchema)
#                     | 'Apply Territories Schema' >> beam.Map(lambda x : beam.Row(territoryid = int(x.territoryid)
#                                                                                  , territoryname = str(x.territoryname)
#                                                                                  , regionid = int(x.regionid)))
#                    | 'Convert to Dictionary' >> beam.Map(lambda row : {"regionid" : row.regionid, "territoryid" : row.territoryid, "territoryname" : row.territoryname})
#                     | SqlTransform("""
#                         SELECT regionid, territoryname as name, territoryid 
#                         FROM PCOLLECTION
#                         """)
#                    | 'Split Territories' >> beam.ParDo(TerritorySplitSchema()).with_output_types(TerritorySchema)
#                     | SqlTransform("""
#                         SELECT regionid, count(*) as territories
#                         FROM PCOLLECTION
#                         GROUP BY regionID
#                         ORDER BY territories DESC
#                         """)
#                    | 'Convert to dictionary' >> beam.Map(lambda row : {"regionid": row.regionid, "territories": row.territories})
                    
#             })
                  )

#     regions | 'Print regions' >> beam.Map(print)
    territories | 'Print territories' >> beam.Map(print)

print('Done')


In [ ]:
# with beam.Pipeline() as pipeline:
#     _ = (
#         pipeline
#         | beam.io.ReadFromPubSub(
#             topic='projects/pubsub-public-data/topics/taxirides-realtime',
#             timestamp_attribute="ts").with_output_types(bytes)
#         | "Parse JSON payload" >> beam.Map(json.loads)
#         # Use beam.Row to create a schema-aware PCollection
#         | "Create beam Row" >> beam.Map(
#             lambda x: beam.Row(
#                 ride_status=str(x['ride_status']),
#                 passenger_count=int(x['passenger_count'])))
#         # SqlTransform will computes result within an existing window
#         | "15s fixed windows" >> beam.WindowInto(beam.window.FixedWindows(15))
#         # Aggregate drop offs and pick ups that occur within each 15s window
#         | SqlTransform(
#             """
#              SELECT
#                ride_status,
#                COUNT(*) AS num_rides,
#                SUM(passenger_count) AS total_passengers
#              FROM PCOLLECTION
#              WHERE NOT ride_status = 'enroute'
#              GROUP BY ride_status""")
#         # SqlTransform yields python objects with attributes corresponding to
#         # the outputs of the query.
#         # Collect those attributes, as well as window information, into a dict
#         | "Assemble Dictionary" >> beam.Map(
#             lambda row,
#             window=beam.DoFn.WindowParam: {
#                 "ride_status": row.ride_status,
#                 "num_rides": row.num_rides,
#                 "total_passengers": row.total_passengers,
#                 "window_start": window.start.to_rfc3339(),
#                 "window_end": window.end.to_rfc3339()
#             })
#         | "Convert to JSON" >> beam.Map(json.dumps)
#         | "UTF-8 encode" >> beam.Map(lambda s: s.encode("utf-8"))
#         | beam.Map(print)
#         #| beam.io.WriteToPubSub(topic=output_topic))
#     )


# if __name__ == '__main__':
#   logging.getLogger().setLevel(logging.INFO)
#   import argparse

#   parser = argparse.ArgumentParser()
#   parser.add_argument(
#       '--output_topic',
#       dest='output_topic',
#       required=True,
#       help=(
#           'Cloud PubSub topic to write to (e.g. '
#           'projects/my-project/topics/my-topic), must be created prior to '
#           'running the pipeline.'))
#   known_args, pipeline_args = parser.parse_known_args()

#   run(known_args.output_topic, pipeline_args)


In [ ]:
lookuptable = [{'regionid':1, 'regionname':'North'}, {'regionid':2, 'regionname':'South'}]
lookup = {e['regionid'] : e['regionname'] for e in lookuptable }
print(lookup)

In [ ]:
import apache_beam as beam
from apache_beam.pvalue import AsIter, AsSingleton, AsList, AsDict
from apache_beam.io import ReadFromText, WriteToText
from apache_beam.io import ReadFromAvro, WriteToAvro
from collections import namedtuple
from apache_beam import coders
from apache_beam.typehints.decorators import with_output_types


class Region:
    def __init__(self, regionid, regionname):
        self.regionid = regionid
        self.regionname = regionname
        
    def __str__(self):
        return f'{self.regionid}|{self.regionname}'

#     def encode(self, o):
#         """Encode to bytes with a trace that coder was used."""
#         # Our encoding prepends an 'x:' prefix.
#         return b'x:%s' % o.encode('utf-8')

#     def decode(self, s):
#         # To decode, we strip off the prepended 'x:' prefix.
#         s = s.decode('utf-8')
#         #assert s[0:2] == 'x:'
#         params = s[0:2].split('|')
#         return Region(*params)

#     def is_deterministic(self):
#         # Since coded Player objects are used as keys below with
#         # beam.CombinePerKey(sum), we require that this coder is deterministic
#         # (i.e., two equivalent instances of the classes are encoded into the same
#         # byte string) in order to guarantee consistent results.
#         return True
    
class RegionSplitClass(beam.DoFn):
    def process(self, element):
        regionid, regionname = element.split(',')
        yield Region(int(regionid), regionname.title())

# class RegionCoder(coders.Coder):
#   """A custom coder for the RegionSchema"""
#   def encode(self, o):
#     """Encode to bytes with a trace that coder was used."""
#     # Our encoding prepends an 'x:' prefix.
#     return b'x:%s' % o.encode('utf-8')

#   def decode(self, s):
#     # To decode, we strip off the prepended 'x:' prefix.
#     s = s.decode('utf-8')
#     #assert s[0:2] == 'x:'
#     params = s[0:2].split('|')
#     return Region(*params)

#   def is_deterministic(self):
#     # Since coded Player objects are used as keys below with
#     # beam.CombinePerKey(sum), we require that this coder is deterministic
#     # (i.e., two equivalent instances of the classes are encoded into the same
#     # byte string) in order to guarantee consistent results.
#     return True
# coders.registry.register_coder(Region, RegionCoder)

# @with_output_types(typing.Tuple[Region, int])
# def get_regions(descriptor):
#   name, points = descriptor.split(',')
#   return Player(name), int(points)


# RegionSchema = namedtuple("RegionSchema", ("regionid", "regionname"))
# class RegionSplitSchema(beam.DoFn):
#     def process(self, element):
#         regionid, regionname = element.split(',')
#         yield RegionSchema(int(regionid), regionname.title())

class RegionSplitDict(beam.DoFn):
    def process(self, element):
        regionid, regionname = element.split(',')
        yield {'regionid': int(regionid), 'regionname': regionname.title()}


class TerritorySplit(beam.DoFn):
    # split territory into KV pair of (regionid, (territoryid, territoryname))
    def process(self, element):
        territoryid, territoryname, regionid = element.split(',')
        yield(int(territoryid), territoryname, int(regionid))
#        yield (int(regionid), (territoryid, territoryname.title())) 
        
                
def lookup_region(left, right):
    territoryid, territoryname, regionid = left
    yield territoryid, territoryname, regionid
#    yield (territoryid, territorynme, regionid, right.get(regionid, 'No Region'))


class LookupRegion(beam.DoFn):
    def process(self, element, lookuptable = [{'regionid':1, 'regionname':'North'}, {'regionid':2, 'regionname':'South'}]):
#        yield element
        territoryid, territoryname, regionid = element
        lookup = {e['regionid'] : e['regionname'] for e in lookuptable }
        yield(territoryid, territoryname, regionid, lookup.get(regionid, 'No Region'))
# #        yield (int(regionid), (territoryid, territoryname.title())) 


# def dummy(element):
#     return element
# #     regionid = element[0]
# #     territoryid, territoryname = element[1]
# #     return (territoryid, territoryname, regionid)

with beam.Pipeline() as p:
    regions = (
        p | 'Read Regions' >> ReadFromText('regions.csv')
          | 'Split Regions' >> beam.ParDo(RegionSplitDict())
          #| 'Split Regions' >> beam.ParDo(RegionSplitClass())
#          | 'Print Regions' >> beam.Map(print)
    )

#     regions = {1:"North", 2:"South", 3:"East", 4:"West"}
#     regions = p | 'Create Regions' >> beam.Create([(1, 'North'), (2, 'South')])

    
    territories =  (
        p | 'Read Territories' >> ReadFromText('territories.csv')
          | 'Split Territories' >> beam.ParDo(TerritorySplit())
#          | 'Print Territories' >> beam.Map(print)
    )
    
    join = (
        territories
          #| 'Lookup Region' >> beam.Map(dummy)
#          | 'Lookup Region' >> beam.Map(lookup_region, right = beam.pvalue.AsList(regions))
#        | beam.ParDo(LookupRegion())
        | beam.ParDo(LookupRegion(), lookuptable = beam.pvalue.AsList(regions))
        | beam.Map(print)
    )
        
